In [1]:
# - 검증 세트 제작 -

import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [2]:
from sklearn.model_selection import train_test_split

# test_size = n -> 테스트 세트의 비율을 n으로 조정
# data와 target의 20퍼센트를 추출하여 test_input, test_target 생성
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

# data 중 80퍼센트 비율로 추출된 train_input과 train_target 중 20퍼센트를 추출하여 val_input, val_target(검증 세트)으로 생성
# val_ 로 트레이닝 세트를 튜닝 -> 최적의 파라미터 탐색 -> 최종 모델 훈련 -> 테스트로 적합도 확인
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [3]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [4]:
# - 교차 검증 -

from sklearn.model_selection import cross_validate

# train_input과 train_taget을 5폴드로 나누어 훈련 (5개의 모델 생성) / 1,2,3,4 (훈련세트), 5(검증세트) -> 1,2,3,5 (훈련세트), 4(검증세트) -> ...
# dt = 머신러닝 모델 (결정 트리) / train_input = 모델이 학습할 입력 데이터 (특성 행렬) / train_target = 모델이 학습할 때 사용할 정답 데이터
# 한 폴드에는 train_input과 train_target 데이터가 짝을 이루고 있음 -> 5폴드 모두 학습 후 한 폴드로 나머지 4폴드를 검증 -> 5번 시행
scores = cross_validate(dt, train_input, train_target)
print(scores)
# fit time = 모델 훈련 시간 / score time = 검증 시 소요 시간 / test score = 검증 점수

{'fit_time': array([0.00605392, 0.00751472, 0.00752735, 0.00649858, 0.00700736]), 'score_time': array([0.00100017, 0.        , 0.00100541, 0.00101018, 0.        ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [5]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


In [6]:
from sklearn.model_selection import StratifiedKFold

# cv = 폴드 개수를 지정 / 회귀 : cv = KFold(), 분류 : cv = StraitfiedKFold() -> 안에 있는 클래스들이 골고루 나눠질 수 있도록
# 객체를 지정하면 회귀 or 분류 모델인지 판별 후 cv를 자동 지정
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [7]:
# splitter 객체를 만들어 상세하게 조정 가능
# n_splits = 폴드 개수 지정, shuffle = 랜덤하게 섞어서 사용
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [8]:
# - 하이퍼 파라미터 튜닝 -
# 그리드 서치
# min_impurity_decrease의 값에 따라 max_depth의 값이 변경될 수 있음 -> 따라서 순차적이 아닌 한 번에 값을 찾아줘야 함

from sklearn.model_selection import GridSearchCV

# min_impurity_decrease (매개변수) = 트리의 노드를 분할할 때, 불순도(impurity)가 얼마나 감소해야 분할을 수행할지를 결정하는 임계값
# 일반적으로 트리는 노드를 분할할 때 불순도 감소(impurity decrease)가 큰 방향으로 분할을 시도
# 값이 클수록 더 많은 데이터를 포함하는 노드를 생성하려고 시도하는 트리가 생성, 반대로 값이 작을수록 더 많은 분할 -> 모델 성능 최적화, 과적합 방지
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

# GridSearchCV(머신러닝 모델 객체, 샘플링 객체(불순도 매개변수), 동시 모델 훈련 개수)
# n_jobs = -1, 컴퓨터의 모든 cpu 사용
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

# 상기처럼 5폴드 분할 후 훈련 5번 수행 * 불순도 매개변수 5개 = 25번 훈련
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [9]:
# gs(결정트리 훈련 모델)을 훈련하면서 찾아낸 최적의 하이퍼 파라미터 조합의 모델을 dt에 할당
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [10]:
# gs의 최적의 하이퍼 파라미터 조합의 모델의 불순도를 출력
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [11]:
# cv_results = 모든 교차검증 결과를 담고있는 딕셔너리
# 'mean_test_score' = 모든 교차검증 결과 중 교차검증 후 평균 테스트 세트 점수를 출력하는 key
print(gs.cv_results_)
print(gs.cv_results_['mean_test_score'])

{'mean_fit_time': array([0.01524506, 0.011691  , 0.0109808 , 0.01260242, 0.00691838]), 'std_fit_time': array([0.00596099, 0.00066838, 0.00086293, 0.00647721, 0.00125995]), 'mean_score_time': array([0.00532594, 0.0030993 , 0.00251493, 0.00259776, 0.00271268]), 'std_score_time': array([0.00325777, 0.00078419, 0.00066064, 0.00103039, 0.00087743]), 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'min_impurity_decrease': 0.0001}, {'min_impurity_decrease': 0.0002}, {'min_impurity_decrease': 0.0003}, {'min_impurity_decrease': 0.0004}, {'min_impurity_decrease': 0.0005}], 'split0_test_score': array([0.86923077, 0.87115385, 0.86923077, 0.86923077, 0.86538462]), 'split1_test_score': array([0.86826923, 0.86346154, 0.85961538, 0.86346154, 0.86923077]), 'split2_test_score': array([0.8825794 , 0.87680462, 0.87584216, 0.88161694, 0.8825794 ]), 

In [12]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [13]:
# min_impurity_decrease = 불순도를 0.0001부터 0.001까지 0.0001씩 증가시켜 훈련
# max_depth = 노드 분할 층을 5부터 20까지 1씩 증가시켜 훈련
# min_samples_split = 노드의 샘플 수가 n개 이하가 되면 분할 중지

params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [14]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [15]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [16]:
# - 확률 분포 선택 -

# 균등분포 샘플링
from scipy.stats import uniform, randint

# rgen에 0부터 9까지의 숫자를 할당
rgen = randint(0, 10)
# rgen에 0부터 9까지의 숫자 중 10개를 랜덤으로 선택하여 추가
rgen.rvs(10)

array([6, 8, 6, 8, 6, 6, 6, 1, 8, 9], dtype=int64)

In [17]:
# rgen.rvs(1000) 0부터 9까지의 수를 1000번 할당
# np.unique() = rgen함수에 포함되어있는 숫자를 반환하고 각각 숫자들이 몇 개 포함되어 있는지도 반환
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),
 array([118,  93, 101,  95,  80, 111,  86,  93, 112, 111], dtype=int64))

In [18]:
# 0에서 1사이의 실수를 추출하여 포함
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.591032  , 0.20394511, 0.11034918, 0.27956921, 0.84659011,
       0.49477703, 0.30603814, 0.53445132, 0.20764635, 0.7605034 ])

In [19]:
# - 랜덤 서치 -

# min_samples_leaf = 각 리프노드가 최소로 가져야 할 샘플개수 지정하는 매개변수
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

from sklearn.model_selection import RandomizedSearchCV

# RandomizedSearchCV(머신러닝 모델 객체, 샘플링 객체, 모델 학습 반복횟수, cpu 할당 수, 난수 생성)
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001F05715C890>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001F05715F1D0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001F057030790>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001F05715D690>},
                   random_state=42)

In [20]:
# 최적의 하이퍼 파라미터 출력
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [21]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86


In [22]:
# 복습

gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001F05715C890>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001F05715F1D0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001F057030790>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001F05715D690>},
                   random_state=42)

In [23]:
print(gs.best_params_)
# gs.cv_results 딕셔너리 중 mean_test_score key의 밸류 중 최댓값 출력
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
